# A Guided Tour of Ray Core: Ray's Multiprocessing Pool

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_06_ray_api_calls.ipynb) <br>
⬅️ [Previous notebook](./ex_04_remote_classes_revisited.ipynb) <br>

### Overview

[*Distributed multiprocessing.Pool*](https://docs.ray.io/en/latest/multiprocessing.html) makes it easy to scale existing Python applications that use [`multiprocessing.Pool`](https://docs.python.org/3/library/multiprocessing.html) by leveraging *Ray Actors*. Ray supports running distributed python programs with the **multiprocessing.Pool** API using Ray Actors, each running on a [workder node's](https://docs.ray.io/en/latest/ray-core/actors.html#faq-actors-workers-and-resources) dedicated process, instead of local processes on a single host. This makes it easy to scale existing applications that use regular `multiprocessing.Pool` from a single node to a cluster.

<img src="images/dist_multi_pool.png" width="80%" height="55%">

### Learning objectives
In this this tutorial, you will learn how to use:

 * Ray's replacement for Python's normal `Multiprocessing.pool` library 

Let's have a go ....

In [1]:
import numpy as np
import time
import logging
import ray
from ray.util.multiprocessing import Pool

## Multiprocessing Pool example

The following is a simple Python function with a slight delay added (to make it behave like a more complex calculation).

In [2]:
# This could be some complicated and compute intensive task
def func(x):
    time.sleep(1.5)
    return x ** 2

### Compute prime numbers
A slightly more compute intensvie function to compute prime numbers

In [3]:
# Let's compute some prime numbers between a range  2-->N
def is_prime(n):
    for divisor in range(2, int(n ** 0.5) + 1):
        if n % divisor == 0:
            return 0
    return 1

Let's start Ray

In [4]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.0rc1
Dashboard:,http://127.0.0.1:8265


Now we'll create a *Pool* of Actors and distribute its tasks across a cluster (or across the available cores on a laptop). 

Let's use Ray's drop-in replacement for [multiprocessing pool](https://docs.ray.io/en/latest/multiprocessing.html) uses Ray Actors to distribute the tasks.

In [5]:
# Create a Pool of Actors
pool = Pool() # by default it will create pool == number of cores on the machine

for result in pool.map(func, range(12)):
    time.sleep(2)      # sleep for you to check the dashboard; you should see Actors == num of cpus
    print(result)

0
1
4
9
16
25
36
49
64
81
100
121


In [6]:
pool.terminate()

**NOTE**: The distributed version has the trade-off of initial increased overhead, albeit now it can scale-out horizontally across a cluster. The benefits are more pronounced with a more computationally expensive calculation. In other words, its value is amortized over time with compute-intensive complex operations.

In [7]:
num = 2000000 
lst = list(range(num))
results = []

# Create a Pool of five actors
pool = Pool(5) 

In [8]:
%%time
for result in pool.map(is_prime, lst):
    results.append(result)
print(f"Total number of primes in {num} are {sum(results)}")


Total number of primes in 2000000 are 148935
CPU times: user 3.87 s, sys: 140 ms, total: 4.01 s
Wall time: 4.37 s


In [9]:
pool.terminate()

Let's define a compute intensive class that does some matrix
computation. Consider this to be a compute intenstive task
doing massive tensor transformation or computation.

**NOTE**: This will be your excercise

In [10]:
def task(n):
    # Simulate a long intensive task
    # TODO for your excercise
    
    # May be consider a compute-intensive matrix computation 
    # and return results
    return 

Define a Ray remote task that launches `task()` across a pool of Actors on the cluster. It creates
a pool of `Ray Actors`, each scheduled on a cluster node's worker process. 

On a single node or localhost it will be an Actor per CPU

In [11]:
# A long running task doing the work: collecting data, updating the database, transforming data, etc
# Create an Actor pool of num_pool workers nodes
@ray.remote
def launch_long_running_tasks(num_pool):
    pool = Pool(num_pool) # num_pool of Actors
    results = []
    # Iterate over 50 times in batches of 10
    # TODO, replace func with task() here for the exercise
    for result in pool.map(func, range(1, 51, 10)):
        results.append(result)
        
    # Done so terminate pool
    pool.terminate()
    
    return results

### Create a Actor like supervisor that launches all these remote tasks

Remember we have seen this desing pattern: Supervisor and workers

In [12]:
@ray.remote
class LaunchDistributedTasks:
    def __init__(self, limit=5):
        self._limit = limit

    def launch(self):
        # launch the remote task
        return launch_long_running_tasks.remote(self._limit)

### Create our supervisor actor

In [13]:
launcher = LaunchDistributedTasks.remote()

In [14]:
# The inner get() returns the list of ObjectRef or 
# The second get() fetch the values of each ObjectRef
#
values = ray.get(ray.get(launcher.launch.remote()))
print(f" list of results :{values}")
print(f" Total results: {len(values)}")

 list of results :[1, 121, 441, 961, 1681]
 Total results: 5


In [15]:
ray.shutdown()

### Exercises

1. Can you convert `task()` into a compute-intensive function?
2. Use `task()` in `pool.map(task,....)`
3. (**Optional**)  Explore the CPU bound tasks using different Python and Ray strategies for distribute computing in this [notebook](extra/mp_all_nb.ipynb) in the `extra` directory

### Homework

Read and work through the these references to see the power of Ray multiprocessing over the cluster with multiple nodes, multiple cores. It shines with compute intensive tasks, such as image processing and numerical computations. 
1. [10x Faster Parallel Python Without Python Multiprocessing](https://towardsdatascience.com/10x-faster-parallel-python-without-python-multiprocessing-e5017c93cce1)

<img src ="https://miro.medium.com/max/1200/1*BXnuxGm7vYUh1pfwbTy-XA.jpeg">

### Next step

Let's take a tour of the [Ray APIs and Tips & Tricks](ex_06_ray_api_calls.ipynb).

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_06_ray_api_calls.ipynb) <br>
⬅️ [Previous notebook](./ex_04_remote_classes_revisited.ipynb) <br>